In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install hf_xet
!pip install trl
!pip install --upgrade transformers
!pip install tensorboard
!pip install -U bitsandbytes
!pip install transformers_stream_generator
!pip install rouge_score
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 37.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for transformers_stream_generator: filename=transformers_stream_generator-0.0.5-py3-none-any.whl size=12426 sha256=02820f35b427d595df04371b73005c77bd7a92a5c88597386a29597e92fde4ef
  Stored in directory: /root/.cache/pip/wheels/23/e8/f0/b3c58c12d1ffe60bcc8c7d121115f26b2c1878653edfca48db
Successfully built transformers_stream_generator
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=0e55f914d2fd7f5ae9df42e867e22f7baf866ae427f8f19f21c4c993be5b1be1
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.7 MB/s eta 0:00:00


### Import Packages

In [3]:
import os
from datasets import load_dataset, concatenate_datasets, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
from trl import SFTTrainer
from rouge_score import rouge_scorer
from sacrebleu.metrics import BLEU

### Load Datasets

In [ ]:
# load tuning sets for <Sunrise on the Reaping>
sunrise_train_path = r'/content/drive/MyDrive/06 PG/01 SMU/01 Course/03 Track Elective/02 CS614 Generative AI with Large Language Models/12 Group Project/CS614-Project/fine_tuning/sunrise_train_data.json'
sunrise_val_path = r'/content/drive/MyDrive/06 PG/01 SMU/01 Course/03 Track Elective/02 CS614 Generative AI with Large Language Models/12 Group Project/CS614-Project/fine_tuning/sunrise_val_data.json'
sunrise_test_path = r'/content/drive/MyDrive/06 PG/01 SMU/01 Course/03 Track Elective/02 CS614 Generative AI with Large Language Models/12 Group Project/CS614-Project/fine_tuning/sunrise_test_data.json'

sunrise_train_datasets = Dataset.from_json(sunrise_train_path)
sunrise_val_datasets = Dataset.from_json(sunrise_val_path)
sunrise_test_datasets = Dataset.from_json(sunrise_test_path)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# load tuning sets for <All Fours>
allfours_train_path = r'/content/drive/MyDrive/06 PG/01 SMU/01 Course/03 Track Elective/02 CS614 Generative AI with Large Language Models/12 Group Project/CS614-Project/fine_tuning2/reasoned_qa_output/allfours_train_data.json'
allfours_val_path = r'/content/drive/MyDrive/06 PG/01 SMU/01 Course/03 Track Elective/02 CS614 Generative AI with Large Language Models/12 Group Project/CS614-Project/fine_tuning2/reasoned_qa_output/allfours_val_data.json'
allfours_test_path = r'/content/drive/MyDrive/06 PG/01 SMU/01 Course/03 Track Elective/02 CS614 Generative AI with Large Language Models/12 Group Project/CS614-Project/fine_tuning2/reasoned_qa_output/allfours_test_data.json'

allfours_train_datasets = Dataset.from_json(allfours_train_path)
allfours_val_datasets = Dataset.from_json(allfours_val_path)
allfours_test_datasets = Dataset.from_json(allfours_test_path)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Combine two books together
train_set = concatenate_datasets([sunrise_train_datasets, allfours_train_datasets])
val_set = concatenate_datasets([sunrise_val_datasets, allfours_val_datasets])
test_set = concatenate_datasets([sunrise_test_datasets, allfours_test_datasets])

In [ ]:
train_set

Dataset({
    features: ['Question', 'Answer', 'Reasoning'],
    num_rows: 958
})

### Model Tuning: Gemma-7B

In [5]:
from huggingface_hub import notebook_login
notebook_login()

Load a Reasoning LLM: Qwen 3 - 8B and its specified tokeniser.

In [ ]:
# Load Reasoning LLM: Qwen3-8B
gemma_model = 'google/gemma-7b'
# Load the specified tokenizer for the model to ensure consistency
tokenizer = AutoTokenizer.from_pretrained(gemma_model)

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token

'<pad>'

Quantisation Config: This is an important for QLoRA, and the key difference from LoRA.

In [ ]:
# defines the quantisation settings to reduce the memory footpoint and computational cost of large models and often with minimal impact on performance
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # a key parameter which instructs the system to load the model's weights using 4-bit precision thus reducing the memory usage
    bnb_4bit_quant_type="nf4", # specifies the exact type of 4-bit quantisation to use, which is NF4. This is a specific floating-point format designed for neural networks.
    bnb_4bit_compute_dtype=torch.bfloat16, # specifies the computations involving these weights shouodl be using bfloat16
    bnb_4bit_use_double_quant=False, # do not use double quantisation which quantise the quantisation constants
)

In [ ]:
# load the quantised model
model = AutoModelForCausalLM.from_pretrained(
    gemma_model,
    quantization_config=bnb_config,
    device_map="auto"  # instructs the transformers library to automatically distribute the model's layer cross available decives to optimise memory usage
)
model.config.use_cache = False

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Ensure the quantised model can be properly fine tuned.

In [ ]:
# this step handles the underlying complexities to make sure that even though the model's parameters have been compressed to a lower precision, gradients can be still computed and propagated to the LoRA adapters, thus enabling effective fine-tuning.
# without this step, attempting to directly fine-tune a quantised model with LoRA might lead to compatibility issues or an inability to train correctly
model = prepare_model_for_kbit_training(model)

LoRA Configuration

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
model.print_trainable_parameters()

trainable params: 50,003,968 || all params: 8,587,684,864 || trainable%: 0.5823


In [ ]:
def formatting_prompts_func(examples):
    texts = []
    for question, answer, reasoning_list in zip(examples["Question"], examples["Answer"], examples["Reasoning"]):
        reasoning_str = "\n".join([f"Step {i+1}: {step}" for i, step in enumerate(reasoning_list)])
        formatted_text = (
            f"### Question:\n{question}\n\n"
            f"### Thought Process:\n"
            f"{reasoning_str}\n\n"
            f"### Answer:\n{answer}"
        )
        texts.append(formatted_text)
    return {"text": texts}

In [ ]:
train_dataset_formatted = train_set.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=train_set.column_names
)
validation_dataset_formatted = val_set.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=val_set.column_names
)
test_dataset_formatted = test_set.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=test_set.column_names
)

Map:   0%|          | 0/958 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

In [ ]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    eval_strategy="steps",
    report_to="tensorboard",
    dataloader_num_workers=0,
    max_steps=-1,
)


trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_formatted,
    eval_dataset=validation_dataset_formatted,
    peft_config=lora_config,
    args=training_arguments,  # 训练参数
)

print("\nTrainer Configuration Completed。")

Converting train dataset to ChatML:   0%|          | 0/958 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/958 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/958 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/958 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/120 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



Trainer Configuration Completed。


In [ ]:
print("Model Tuning Start...")
trainer.train()
print("Model Tuning Completed")

Model Tuning Start...


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
10,2.381600,2.495190
20,2.404900,2.453832
30,2.314000,2.386662
40,2.257700,2.272040
50,2.245300,2.095351
60,2.004900,2.000942
70,1.979600,1.921161
80,1.846500,1.873063
90,1.839700,1.822731
100,1.763400,1.791453


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Model Tuning Completed


In [ ]:
# Save LoAR Adapter
output_model_path = "/content/drive/MyDrive/06 PG/01 SMU/01 Course/03 Track Elective/02 CS614 Generative AI with Large Language Models/12 Group Project/QLoRA_Gemma"
trainer.save_model(output_model_path)
print(f"Fine-tuned LoRA LoRA Adapter saved to: {output_model_path}")

Fine-tuned LoRA LoRA Adapter saved to: /content/drive/MyDrive/06 PG/01 SMU/01 Course/03 Track Elective/02 CS614 Generative AI with Large Language Models/12 Group Project/QLoRA_Gemma


### Evaluations

In [12]:
base_model = "google/gemma-7b"
adapter_path = "/content/drive/MyDrive/06 PG/01 SMU/01 Course/03 Track Elective/02 CS614 Generative AI with Large Language Models/12 Group Project/CS614-Project/fine_tuning/QLoRA_Gemma"

# quantisation
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(base_model)

from peft import PeftModel

model = PeftModel.from_pretrained(model, adapter_path)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
def evaluate_model(model, tokenizer, dataset):
    model.eval()
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    bleu = BLEU()

    all_rouge_scores = {'rouge1': {'fmeasure': 0, 'precision': 0, 'recall': 0},
                        'rouge2': {'fmeasure': 0, 'precision': 0, 'recall': 0},
                        'rougeL': {'fmeasure': 0, 'precision': 0, 'recall': 0}}
    all_bleu_scores = 0
    count = 0

    for example in dataset:
        prompt = example["text"].split("### Answer:")[0] + "### Answer:"
        reference = example["text"].split("### Answer:")[1].strip()

        input_data = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
        input_ids = input_data.input_ids.to(model.device)
        attention_mask = input_data.attention_mask.to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=100,
                num_return_sequences=1,
                pad_token_id=tokenizer.eos_token_id,
                attention_mask = attention_mask
            )

        generated_text = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True).strip()

        rouge_scores = scorer.score(reference, generated_text)
        for key in all_rouge_scores:
            all_rouge_scores[key]['fmeasure'] += rouge_scores[key].fmeasure
            all_rouge_scores[key]['precision'] += rouge_scores[key].precision
            all_rouge_scores[key]['recall'] += rouge_scores[key].recall

        all_bleu_scores += bleu.sentence_score(generated_text, [reference]).score

        count += 1

    avg_rouge_scores = {}
    for key in all_rouge_scores:
        avg_rouge_scores[key] = {metric: score / count for metric, score in all_rouge_scores[key].items()}
    avg_bleu_score = all_bleu_scores / count

    return avg_rouge_scores, avg_bleu_score

print("\nEvaluating on Test Set...")

test_dataset_formatted = test_set.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=test_set.column_names
)
test_rouge, test_bleu = evaluate_model(model, tokenizer, test_dataset_formatted)
print("Test Set Metrics:")
print("ROUGE Scores:", test_rouge)
print("BLEU Score:", test_bleu)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Evaluating on Test Set...


Test Set Metrics:
ROUGE Scores: {'rouge1': {'fmeasure': 0.5446038691557366, 'precision': 0.5760281781506387, 'recall': 0.5261729362789646}, 'rouge2': {'fmeasure': 0.33049114771934546, 'precision': 0.34936879085524636, 'recall': 0.3184926817229733}, 'rougeL': {'fmeasure': 0.47739747365038576, 'precision': 0.504654793608247, 'recall': 0.4611013602169552}}
BLEU Score: 25.719886347968345


In [13]:
torch.cuda.empty_cache()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# The model should already be on the device because of device_map="auto"
# No need for an explicit model.to(device) call if device_map="auto" is used

def chat_with_model(model, tokenizer):
    print("Start chatting with the model. Type 'quit' to exit.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            break

        # Format the user input as a prompt
        prompt = f"### Question:\n{user_input}\n\n### Thought Process:\n"

        # Ensure input_ids are on the correct device
        input_data = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
        input_ids = input_data.input_ids.to(model.device)
        attention_mask = input_data.attention_mask.to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=200, # Adjust based on desired response length
                num_return_sequences=1,
                attention_mask=attention_mask,
                pad_token_id=tokenizer.eos_token_id,
                temperature=0.7, # Adjust for creativity (lower is less creative)
                top_p=0.9,       # Adjust for sampling
                do_sample=True   # Enable sampling
            )

        generated_text = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True).strip()

        # Find the start of the Answer section to only display the answer
        answer_start_index = generated_text.find("### Answer:")
        if answer_start_index != -1:
            # Find the start of the next section (if any)
            next_section_index = generated_text.find("###", answer_start_index + len("### Answer:"))
            if next_section_index != -1:
                generated_answer = generated_text[answer_start_index + len("### Answer:"):next_section_index].strip()
            else:
                generated_answer = generated_text[answer_start_index + len("### Answer:"):].strip()
            print(f"Model: {generated_answer}")
        else:
            # If no explicit Answer section, print the generated text after the Thought Process
            thought_process_end_index = generated_text.find("\n\n### Answer:")
            if thought_process_end_index != -1:
                print(f"Model: {generated_text[:thought_process_end_index].strip()}")
            else:
                 print(f"Model: {generated_text.strip()}")


# Start the chat interface
chat_with_model(model, tokenizer)

Start chatting with the model. Type 'quit' to exit.
You: What items does Haymitch give his mother and brother before leaving for the Games?


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Model: Haymitch gives his mother 'a dozen apples and a couple of oranges' and his brother 'two eggs and a packet of corn cakes'.
You: What is the name of the plant at Plutarch’s conservatory with a faint sweet rotten smell?
Model: The plant at Plutarch’s conservatory with a faint sweet rotten smell is called 'flowering dogwood'.
You: What item does Maysilee use to remove a spike from her cheek?
Model: Maysilee uses her finger to remove a spike from her cheek, wincing in pain.
You: What is the item Maysilee knuckle-rolls on the train, and what is it made of?
Model: Maysilee is 'knuckle-roll[ing] a piece of white dough' on the train, made from 'white flour'.
You: What are the squirrel mutts’ primary movement method and coat color?
Model: The squirrel mutts primarily move by 'bouncing from branch to branch' and have 'orange and yellow' coats.
You: quit


In [14]:
import json

mcq_data_path = "/content/drive/MyDrive/06 PG/01 SMU/01 Course/03 Track Elective/02 CS614 Generative AI with Large Language Models/12 Group Project/CS614-Project/mcq_evaluation/qa_data.json"

# Load the MCQ data
with open(mcq_data_path, 'r') as f:
    mcq_data = json.load(f)

In [18]:
def evaluate_mcq(model, tokenizer, mcq_data):
    model.eval()
    correct_count = 0
    total_count = 0

    for item in mcq_data:
        question = item["question"]
        options = "\n".join([f"{key}. {value}" for key, value in item["options"].items()])
        correct_answer_key = item["answer"]

        prompt = f"### Question:\n{question}\n{options}\n\n### Thought Process:\n"

        input_data = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
        input_ids = input_data.input_ids.to(model.device)
        attention_mask = input_data.attention_mask.to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=200,
                num_return_sequences=1,
                pad_token_id=tokenizer.eos_token_id,
                attention_mask=attention_mask,
                temperature=0.1, # Keep temperature low for deterministic answers,
                do_sample=False  # Disable sampling for deterministic answers
            )

        generated_text = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True).strip()

        # Extract the predicted answer key from the generated text
        # This part might need adjustment based on the exact output format of your model
        # Look for the part after "### Answer:" and try to match it to the option keys (A, B, C, D)
        predicted_answer_key = None
        answer_start_index = generated_text.find("### Answer:")
        if answer_start_index != -1:
            answer_content = generated_text[answer_start_index + len("### Answer:"):].strip()
            # Simple heuristic: look for a single character (like A, B, C, D) at the start
            if len(answer_content) > 0 and answer_content[0] in item["options"].keys():
                predicted_answer_key = answer_content[0]
            elif len(answer_content) > 1 and answer_content[1] == '.' and answer_content[0] in item["options"].keys():
                 predicted_answer_key = answer_content[0]


        print(f"Question: {question}")
        print(f"Predicted Answer Key: {predicted_answer_key}")
        print(f"Correct Answer Key: {correct_answer_key}")


        if predicted_answer_key is not None and predicted_answer_key == correct_answer_key:
            correct_count += 1
            print("Result: Correct")
        else:
            print("Result: Incorrect")

        total_count += 1
        print("-" * 20)


    accuracy = (correct_count / total_count) * 100 if total_count > 0 else 0
    return accuracy

print("\nEvaluating on MCQ data...")
mcq_accuracy = evaluate_mcq(model, tokenizer, mcq_data)
print(f"\nMCQ Accuracy: {mcq_accuracy:.2f}%")

torch.cuda.empty_cache()

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Evaluating on MCQ data...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Question: In the novel 'All Fours', what reason does the narrator's neighbor, Brian, give in a note for his concern?
Predicted Answer Key: None
Correct Answer Key: B
Result: Incorrect
--------------------


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

Question: After the Games, what does Haymitch see on a wall in a back alley that confirms Lenore Dove's rebellious activities?
Predicted Answer Key: A
Correct Answer Key: C
Result: Incorrect
--------------------


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Question: What does Maysilee use to create a makeshift glue to repair Kerna's broken sunflower token?
Predicted Answer Key: None
Correct Answer Key: B
Result: Incorrect
--------------------


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Question: What is the first muttation Haymitch successfully fights off using his flint striker and a gas plant?
Predicted Answer Key: None
Correct Answer Key: C
Result: Incorrect
--------------------
Question: What is the final 'poster' Haymitch creates in the arena?
Predicted Answer Key: None
Correct Answer Key: D
Result: Incorrect
--------------------

MCQ Accuracy: 8.00%
